Modele générant les embeddings pour VGG16; en cours de finalisation

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam,Adagrad

from keras.applications.vgg16 import VGG16
from keras.applications import VGG19
from keras.applications import EfficientNetB0




from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

from RktnChallenge.RktnModel import ModelTrainer


# PARAMS  de base
PATH = "images/crop224/"
img_size = 224
depth = 3
num_categories =27

In [ ]:
preprocessing_function = tf.keras.applications.vgg16.preprocess_input
trainer = ModelTrainer("data.csv",PATH)
trainer.loadData(PATH=PATH,imgsize=224,batchsize=32,aumgentImages=False,
                preprocessing_function=preprocessing_function,customized_data=True)

In [ ]:
input = tf.keras.Input(shape=(img_size,img_size,3))
net = VGG16(weights='imagenet', include_top=False,input_tensor = input) 



for layer in net.layers[:-2]:
    layer.trainable = False
    
gap = tf.keras.layers.GlobalAveragePooling2D()(net.output)
norm2 = tf.keras.layers.BatchNormalization()(gap)

dense1 = tf.keras.layers.Dense(512, activation='relu')(norm2)
dropout1 = tf.keras.layers.Dropout(0.3)(dense1)

dense2 = tf.keras.layers.Dense(124, activation='relu')(dropout1)
dropout2 = tf.keras.layers.Dropout(0.3)(dense2)

output = tf.keras.layers.Dense(num_categories, activation='softmax')(dropout2)




base_model = tf.keras.Model(net.input, outputs = output)
base_model.compile(loss=['sparse_categorical_crossentropy'], optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])


In [ ]:
trainer.train(model=base_model, epochs = 10)